In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Q1
print("TensorFlow Version:", tf.__version__)
print("Keras Version:", tf.keras.__version__)

# Q2
url = "https://raw.githubusercontent.com/nareshbhat/wine-quality-binary-classification/master/winequality-white.csv"
wine_df = pd.read_csv(url)
print("Dataset Dimensions:", wine_df.shape)

# Q3
print("Null Values:\n", wine_df.isnull().sum())
categorical_vars = ['quality']
wine_df[categorical_vars] = wine_df[categorical_vars].astype('category')
wine_df = pd.get_dummies(wine_df, columns=categorical_vars, drop_first=True)

# Q4
features = wine_df.drop(columns=['quality_good'])
target = wine_df['quality_good']

# Q5
X_train, X_temp, y_train, y_temp = train_test_split(features, target, test_size=0.3, stratify=target, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

# Q6
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Q7
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Q8
model.add(Dense(1, activation='sigmoid'))

# Q9
tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=False)

# Q10
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Q11
model_checkpoint_callback = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True)

# Q12
model.summary()

# Q13
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Q14
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=64,
                    validation_data=(X_val_scaled, y_val),
                    callbacks=[tensorboard_callback, early_stopping_callback, model_checkpoint_callback])

# Q16
model_params = model.get_weights()

# Q17
history_df = pd.DataFrame(history.history)

# Q18
plt.plot(history_df['accuracy'], label='Accuracy')
plt.plot(history_df['val_accuracy'], label='Validation Accuracy')
plt.plot(history_df['loss'], label='Loss')
plt.plot(history_df['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Metric')
plt.legend()
plt.show()

# Q19
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
